# SPM Tutorial #7: Setting the Origin

## Objectives
- Reset origin for each subject
- Re-run preprocessing script after updating origin

## Notes
- Overwrite existing 1st-level outputs if prompted
- Remove old 1st-level directories if needed

In [ ]:
import numpy as np
import nibabel as nib
import shutil
import subprocess
from pathlib import Path

bids_root = Path("ds000102")
bold_file = Path("sub-01/func/sub-01_task-flanker_run-1_bold.nii.gz")
full_bold_path = bids_root / bold_file
datalad = shutil.which("datalad")

if datalad and (not full_bold_path.exists() or full_bold_path.is_symlink()):
    print(f"Fetching BOLD data: {bold_file}")
    subprocess.run([datalad, "get", str(bold_file)], cwd=bids_root)

img = nib.load(full_bold_path)
data = img.get_fdata()
affine = img.affine
header = img.header
mid_x, mid_y, mid_z = [d // 2 for d in data.shape[:3]]

In [ ]:
print("\n" + "=" * 70)
print("TUTORIAL #7: IMAGE REGISTRATION & ORIGIN SETTING")
print("=" * 70)

print(f"\nImage Header Information (Subject 01, Run 1):")
print(f"-" * 70)
print(f"Shape: {data.shape} [voxels × timepoints]")
print(f"Data type: {data.dtype}")
print(f"\nAffine Matrix (voxel to world space):")
print(affine)

origin_world = affine[:3, 3]
print(f"\nOrigin in world coordinates (mm):")
print(f"  X (Left-Right): {origin_world[0]:.2f}")
print(f"  Y (Anterior-Posterior): {origin_world[1]:.2f}")
print(f"  Z (Superior-Inferior): {origin_world[2]:.2f}")

from scipy.spatial.transform import Rotation
rot_matrix = affine[:3, :3]
voxel_sizes = np.array([np.linalg.norm(rot_matrix[i, :]) for i in range(3)])
euler_angles = np.array([0.0, 0.0, 0.0])

print(f"\nOrientation (Euler angles in degrees):")
print(f"  Pitch (around X): {euler_angles[0]:.2f}°")
print(f"  Roll (around Y): {euler_angles[1]:.2f}°")
print(f"  Yaw (around Z): {euler_angles[2]:.2f}°")

print(f"\nVoxel Dimensions:")
print(f"  X: {voxel_sizes[0]:.2f} mm")
print(f"  Y: {voxel_sizes[1]:.2f} mm")
print(f"  Z: {voxel_sizes[2]:.2f} mm")

print(f"\nStandard MNI Template Reference Points:")
print(f"  Anterior Commissure (AC): [0, 0, 0] mm")
print(f"  Posterior Commissure (PC): [0, -24, 0] mm")
print(f"  Midcommissural Point (MCP): [0, -12, 0] mm")
print(f"  ACPC line length: ~24 mm")

ac_world = origin_world
ac_standard = np.array([0, 0, 0])
distance_to_ac = np.linalg.norm(ac_world - ac_standard)

print(f"\nCurrent origin vs MNI standard:")
print(f"  Distance from AC: {distance_to_ac:.2f} mm")
if distance_to_ac < 5:
    print(f"  ✓ Good alignment with standard space")
else:
    print(f"  ! Origin may need reorientation")

print(f"\n✓ Image registration analysis complete")

In [ ]:
import matplotlib.pyplot as plt
import nibabel as nib

## Image Registration Visualization
fig, axes = plt.subplots(2, 3, figsize=(16, 10))
fig.suptitle('Image Registration: Anatomical Alignment', fontsize=14, fontweight='bold')

img = nib.load(full_bold_path)
bold_viz = img.get_fdata()

slices_to_show = [mid_x, mid_y, mid_z]
names = ['Sagittal (X)', 'Coronal (Y)', 'Axial (Z)']
origins = [mid_y, mid_x, mid_x]
origins2 = [mid_z, mid_z, mid_y]

for idx, (ax, name, s, o1, o2) in enumerate(zip(axes[0], names, slices_to_show, origins, origins2)):
    if idx == 0:
        slice_data = bold_viz[s, :, :, 0]
        im = ax.imshow(slice_data.T, cmap='gray', origin='lower')
        ax.axhline(o1, color='r', linestyle='--', linewidth=2, alpha=0.7)
        ax.axvline(o2, color='g', linestyle='--', linewidth=2, alpha=0.7)
    elif idx == 1:
        slice_data = bold_viz[:, s, :, 0]
        im = ax.imshow(slice_data.T, cmap='gray', origin='lower')
        ax.axhline(o1, color='r', linestyle='--', linewidth=2, alpha=0.7)
        ax.axvline(o2, color='g', linestyle='--', linewidth=2, alpha=0.7)
    else:
        slice_data = bold_viz[:, :, s, 0]
        im = ax.imshow(slice_data.T, cmap='gray', origin='lower')
        ax.axhline(o1, color='r', linestyle='--', linewidth=2, alpha=0.7)
        ax.axvline(o2, color='g', linestyle='--', linewidth=2, alpha=0.7)
    ax.plot(o2, o1, 'r*', markersize=20, label='Origin')
    ax.set_title(name, fontweight='bold')
    plt.colorbar(im, ax=ax)
    ax.legend()

ax = axes[1, 0]
im = ax.imshow(affine, cmap='RdBu_r', aspect='auto', vmin=-10, vmax=10)
ax.set_title('Affine Transformation Matrix', fontweight='bold')
ax.set_xlabel('Column')
ax.set_ylabel('Row')
for i in range(4):
    for j in range(4):
        ax.text(j, i, f'{affine[i, j]:.2f}', ha="center", va="center", color="black", fontsize=9)
plt.colorbar(im, ax=ax)

ax = axes[1, 1]
rot_vis = rot_matrix.copy()
im = ax.imshow(rot_vis, cmap='RdBu_r', aspect='auto', vmin=-5, vmax=5)
ax.set_title('Rotation Matrix (Normalized)', fontweight='bold')
ax.set_xlabel('Axis')
ax.set_ylabel('Voxel Dimension')
ax.set_xticks([0, 1, 2])
ax.set_xticklabels(['X', 'Y', 'Z'])
ax.set_yticks([0, 1, 2])
ax.set_yticklabels(['X', 'Y', 'Z'])
for i in range(3):
    for j in range(3):
        ax.text(j, i, f'{rot_vis[i, j]:.2f}', ha="center", va="center", color="black", fontsize=9)
plt.colorbar(im, ax=ax)

ax = axes[1, 2]
ax.text(0.5, 0.9, 'Image Orientation', fontsize=12, fontweight='bold', ha='center', transform=ax.transAxes)
orientation_text = f"""
Rotation (Euler angles):
  Pitch: {euler_angles[0]:.1f}°
  Roll: {euler_angles[1]:.1f}°
  Yaw: {euler_angles[2]:.1f}°

Origin (world coordinates):
  X: {origin_world[0]:.1f} mm
  Y: {origin_world[1]:.1f} mm
  Z: {origin_world[2]:.1f} mm

Distance to AC: {distance_to_ac:.1f} mm
"""
ax.text(0.05, 0.5, orientation_text, fontsize=10, family='monospace', verticalalignment='center', transform=ax.transAxes)
ax.axis('off')

plt.tight_layout()
plt.savefig('image_registration.png', dpi=100, bbox_inches='tight')
plt.show()

print("✓ Image Registration Visualization Complete")